In [2]:
import cv2 as cv
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#pip install lxml
from lxml import objectify
from os import listdir
import xml.etree.ElementTree as ET

In [34]:
filepath='C:/Users/Miranda/Documents/Norditech_project/annotations.xml'
path = 'C:/Users/Miranda/Documents/Norditech_project/label-project/data_2/' 

In [20]:
tree = ET.parse('annotations.xml')
root = tree.getroot()
root.tag

'annotations'

In [36]:
images=[]
for  _,_, files in os.walk(path + "images"):
    for name in files:
        images.append(name)
print(images)

['0144f887-d7b5-4bb0-bf17-283804071595.png', '028554f4-c7d8-4ab6-8b74-a6a6ac7c3f90.png', '0a11388a-a2f8-4ff4-bf5c-7e4c5f297751.png', '1031b0c7-eec4-4fa7-82cd-690e51b2cac1.png', '12cf2d0f-436f-4698-a501-2521e22264d1.png', '1613ff33-03cc-4d10-afb0-55ebf0722042.png', '18d7ad1f-0cfd-4563-9c81-9b41e5413f0d.png', '1ac3210d-e668-4946-9f0b-2c05b26a3653.png', '1c99c1cc-8b05-43ff-838c-d4da6c63a02a.png', '200c05c0-17d2-45d6-9c14-6abb5ef286c7.png', '217fc145-ea83-4b82-9c5c-d4d53a3d982a.png', '23355f0f-1bd9-4339-aa95-db16fb7436ed.png', '24f0324c-84d1-4ded-a279-bc3e1e89da72.png', '27898689-d8c6-4b37-9b0a-416903f3457f.png', '2fe43136-85b5-44d9-9540-dcdf3c3c6916.png', '35fcebc6-b85a-4306-bacf-5d064debdd71.png', '373c1177-4ba1-4648-b7e2-b247ec0fe263.png', '4010f7ba-5de9-4dc0-9688-da8f4a50eefc.png', '41f85a89-becc-42d0-8f37-f55f19c6f976.png', '42809a8b-9748-4ff7-b657-68bbea6b3507.png', '44289653-1d6a-46cf-9c6f-17dc65b1f93e.png', '477ae9eb-8f9b-4c76-8443-170e795aa806.png', '48806201-978a-480d-9828-9feed3

In [ ]:
for polygon in image.getchildren():
        polygon_points = [e.split(',') for e in polygon.attrib['points'].split(';')]
        polygon_points = [(int(float(x)), int(float(y))) for x, y in ploygon_points]
        polygons.append(polygon_points)
        bounding_box = cv.boundingRect(np.array(ploygon_points, np.int32))
        bounding_boxes.append(bounding_box)

In [30]:
from xml.dom import minidom
xmldoc = minidom.parse(filepath)
itemlist = xmldoc.getElementsByTagName('image')
images=[]
#for i in root.iter('image'):
    for i in itemlist:
        img = i.attributes["name"].value
        #images.append(img)

print(images)

[]


In [15]:
for img in images:
    #xml_data = objectify.parse(filepath)
    #root = xml_data.getroot()
    #image = root.xpath(f"//image[@name='{img}.png']")
    i = cv.imread(img ) #is without the png
    mask = np.zeros(i.shape[0:2], dtype=np.uint8)
    region = (591.00, 2785.09), (995.60, 2811.70), (994.70, 2847.10), (591.72, 2818.13)
    points = np.array(region, np.int32)
    #smooth region
    cv.drawContours(mask, [points], -1, (255, 255, 255), -1, cv.LINE_AA)
    #not so smooth region
    res = cv.bitwise_and(img,img,mask = mask)
    rect = cv.boundingRect(points) # returns (x,y,w,h) of the rect
    cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
    wbg = np.ones_like(img, np.uint8)*255
    cv.bitwise_not(wbg,wbg, mask=mask)
    dst = wbg+res
    cv.imshow('Original',img)
    cv.imshow("Mask",mask)
    cv.imshow("Cropped", cropped )
    cv.imshow("Samed Size Black Image", res)
    cv.imshow("Samed Size White Image", dst)
    cv.waitKey(0)
    cv.imwrite(f'C:/Users/Miranda/Documents/Norditech_project/label-project/data_2/polygon/crop_{img}.png',cropped)
#cv.destroyAllWindows()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
labels = []
i = 0
for child in tree.getroot():
    if child.tag == 'image':
        img = cv.imread('dataset/data_2/images/' + child.attrib['name'], cv.IMREAD_GRAYSCALE)
        for polygon in child:
            points = [[int(float(p)) for p in e.split(',')] for e in polygon.attrib['points'].split(';')]
            points = np.array(points, dtype=np.int32)
            mask = np.zeros(img.shape[0:2], dtype=np.uint8)
            cv.drawContours(mask, [points], -1, (255, 255, 255), -1, cv.LINE_AA)
            masked_img = img & mask
            rect = cv.boundingRect(points)
            out = masked_img[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]
            labels.append(polygon[0].text)
            cv.imwrite(f'dataset/data_2/poly_crops/{str(i).zfill(5)}.png', out)
            i += 1
df = pd.DataFrame(labels)
pd.to_csv('dataset/data_2/poly_labels.csv', header=False, index=False)

,0
0,test
1,test2
2,test3
